In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [3]:
data = pd.read_csv("churn_modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## preprocess data
## drop irrelevant data
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [6]:
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
#encode one categorial variable
lable_encoder_gender=LabelEncoder()
data['Gender']=lable_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [10]:
#onehot encode geography
from sklearn.preprocessing import OneHotEncoder
one_hote_geo=OneHotEncoder()
geo_encoder=one_hote_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [12]:
one_hote_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [14]:
geo_df=pd.DataFrame(geo_encoder.toarray(),columns=one_hote_geo.get_feature_names_out(['Geography']))
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [20]:
## combine one hot encoded column with data
data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [21]:
## save encoder and scaler

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(lable_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hote_geo,file)


In [23]:
# divide the dataset into independent and dependent feature
x=data.drop('Exited',axis=1)
y=data['Exited']

## split the data in training and testing dataset
X_train,X_test,Y_train,Y_test= train_test_split(x,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [24]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [25]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [31]:
## build our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## !st Hidden layer
    Dense(32,activation='relu'), ## 2nd hidden layer
    Dense(1,activation='sigmoid') ## output layer
])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [38]:
##Compile model
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])

In [39]:
## set up the Tensorboard

log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_Callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [40]:
## set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [41]:
## Train modle
history=model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,
    callbacks=[tensorflow_Callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 0.4492 - accuracy: 0.7991 - val_loss: 0.3840 - val_accuracy: 0.8305
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3793 - accuracy: 0.8456 - val_loss: 0.3559 - val_accuracy: 0.8500
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3557 - accuracy: 0.8534 - val_loss: 0.3454 - val_accuracy: 0.8545
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3455 - accuracy: 0.8605 - val_loss: 0.3436 - val_accuracy: 0.8565
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3405 - accuracy: 0.8586 - val_loss: 0.3479 - val_accuracy: 0.8600
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3371 - accuracy: 0.8620 - val_loss: 0.3422 - val_accuracy: 0.8590
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3331 - accuracy: 0.8639 - val_loss: 0.3433 - val_accuracy: 0.86

In [42]:
model.save('model.h5')

d:\python\Flipkart_scrapping\first project\Ann clasification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [43]:
## Load Tensorboard Exctension
%load_ext tensorboard

In [51]:
%tensorboard --logdir logs/fit20260108-203205

Reusing TensorBoard on port 6008 (pid 3692), started 0:25:30 ago. (Use '!kill 3692' to kill it.)

In [52]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [54]:
### load trained mode, scaler pickle and onehot
model=load_model('model.h5')

##load the encoder as scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    lable_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [77]:
## example data

input = {
    'Creditscore':600,
    'Geography' :'France',
    'Gender' :'Male',
    'age':40,
    'tenure':3,
    'Balance' :60000,
    'NumofProducts' :2,
    'HasCrCard' :1,
    'IsActiveMember':1,
    'EstimatedSalary':50000



}

In [78]:
geo_encoder = lable_encoder_geo.transform([[input['Geography']]]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder,columns=lable_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

d:\python\Flipkart_scrapping\first project\Ann clasification\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [79]:
input_df=pd.DataFrame([input])
input_df

,Creditscore,Geography,Gender,age,tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [80]:
# Encode  categorical variable
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,Creditscore,Geography,Gender,age,tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [83]:
input_df= pd.concat([input_df.drop("Geography",axis=1),geo_encoder_df],axis=1)
input_df

,Creditscore,Gender,age,tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [87]:
## scaling data

input_scaler=scaler.transform(input_df)
input_scaler

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [88]:
prediction=model.predict(input_scaler)
prediction

1/1 [==============================] - 0s 446ms/step


array([[0.17826055]], dtype=float32)

In [89]:
prediction_proba = prediction[0][0]
prediction_proba

0.17826055

In [90]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')

else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
